In [2]:
# %load_ext autoreload 
# %autoreload 2

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from itertools import zip_longest

varr = {}
varr['main_path'] = "C:\\Users\\jamilah\\Documents\\Github_analysis_PROJECTS\\Time_series_analysis\\Motor_classification\\Motor_classification"  
varr['main_path1'] = "%s\\a_data_standardization" % (varr['main_path'])
varr['main_path2'] = "%s\\b_data_preprocessing" % (varr['main_path'])
varr['main_path3'] = "%s\\c_calculate_metrics" % (varr['main_path'])

import sys
sys.path.insert(1, '%s' % (varr['main_path']))


from c_calculate_metrics.put_scalar_trialdata_into_pandas import *
from c_calculate_metrics.put_timeseries_trialdata_into_pandas import *

from subfunctions.make_a_properlist import *
from subfunctions.confidence_interval import *
from subfunctions.two_sample_stats import *
from subfunctions.semi_automated_gen import *
from subfunctions.my_dropna_python import *
from subfunctions.compare_condi import compare_across_condi, compare_within_condi

df_scalarmetics_exp = put_scalar_trialdata_into_pandas(varr)
df_timeseries_exp = put_timeseries_trialdata_into_pandas(varr)

C:\Users\jamilah\Documents\Github_analysis_PROJECTS\Time_series_analysis\Motor_classification\Motor_classification\c_calculate_metrics\put_scalar_trialdata_into_pandas.py:44: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  dat = np.array(dat)
C:\Users\jamilah\Documents\Github_analysis_PROJECTS\Time_series_analysis\Motor_classification\Motor_classification\c_calculate_metrics\put_scalar_trialdata_into_pandas.py:49: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  X = np.array(X)
C:\Users\jamilah\AppData\Roaming\Python\Python39\site-packages\numpy\core\_asarr

# Testing if data is correct : 

In [3]:
print('shape of df_scalarmetics_exp[rot] : ', df_scalarmetics_exp['rot'].shape)
df_scalarmetics_exp['rot'].tail()

shape of df_scalarmetics_exp[rot] :  (442, 13)


,subject,tr,ss,ax,new3_ind_st,new3_ind_end,trnum_org,SSQ_b4,SSQ_af,SSQ_diff,FRT_em,res_type,TR
437,17,16,-1,2,5066,5252,28.0,"[0, 1, 0, 1]","[2, 3, 2, 5]","[-2, -2, -2, -4]",3.908,7,18.499999
438,17,17,2,0,5627,5790,31.0,"[0, 1, 0, 1]","[2, 3, 2, 5]","[-2, -2, -2, -4]",2.66,7,16.199999
439,17,18,-2,0,5974,6136,33.0,"[0, 1, 0, 1]","[2, 3, 2, 5]","[-2, -2, -2, -4]",1.668,6,16.100001
440,17,19,0,1,6141,6367,34.0,"[0, 1, 0, 1]","[2, 3, 2, 5]","[-2, -2, -2, -4]",8.04,10,7.8
441,17,20,-1,0,7475,7643,41.0,"[0, 1, 0, 1]","[2, 3, 2, 5]","[-2, -2, -2, -4]",5.256,7,16.7


In [ ]:
print('shape of df_scalarmetics_exp[trans] : ', df_scalarmetics_exp['trans'].shape)
df_scalarmetics_exp['trans'].tail()

In [4]:
print('shape of df_timeseries_exp[rot] : ', df_timeseries_exp['rot'].shape)
df_timeseries_exp['rot'].tail()

shape of df_timeseries_exp[rot] :  (86006, 19)


,subject,tr,ss,ax,dp,time,res_type,SIGCOM_ax0,SIGCOM_ax1,SIGCOM_ax2,SIG_ax0,SIG_ax1,SIG_ax2,JOY_ax0,JOY_ax1,JOY_ax2,NOISE_ax0,NOISE_ax1,NOISE_ax2
86001,17.0,20.0,-1.0,0.0,163.0,16.300000,7.0,-1.453208,0.804159,-4.699437,-1.514817,0.803347,-4.719625,0.301392,-0.082375,-4.927108,0.850650,0.0,-4.927108
86002,17.0,20.0,-1.0,0.0,164.0,16.400000,7.0,-1.391906,0.804159,-4.699425,-1.461697,0.803352,-4.719593,0.333524,-0.082369,-4.917930,0.902199,0.0,-4.917930
86003,17.0,20.0,-1.0,0.0,165.0,16.500001,7.0,-1.313130,0.804159,-4.699425,-1.382446,0.807940,-4.719292,0.356304,-0.086946,-4.908213,0.953649,0.0,-4.908213
86004,17.0,20.0,-1.0,0.0,166.0,16.599999,7.0,-1.221655,0.804159,-4.699425,-1.284506,0.807370,-4.719829,0.362870,-0.089799,-4.897957,1.004994,0.0,-4.897957
86005,17.0,20.0,-1.0,0.0,167.0,16.700000,7.0,-1.125844,0.804159,-4.699425,-1.179229,0.805081,-4.719719,0.364322,-0.088796,-4.887165,1.056230,0.0,-4.887165


In [ ]:
print('shape of df_timeseries_exp[trans] : ', df_timeseries_exp['trans'].shape)
df_timeseries_exp['trans'].head()

In [ ]:
def category_stats(incon, inner_name_list, outer_name_list, ax_all_vals, outer2_name, cou, final_temp):

    num_of_tests = 2
    rshift = 0.12
    
    # Vector retrival
    for i in range(len(outer_name_list)):
        vec[i] = semi_automated_gen(incon, outer_name_list[i], ax_all_vals, inner_name_list, outer_name_list)
    
    # Statistical tests
    df_res = two_sample_stats(vec[0], vec[1], num_of_tests)
    
    # Saving stats in a DataFrame
    col0 = pd.Series('%s %s %s' % (outer2_name, incon, outer_name_list))  # string
    temp_cur = pd.concat([col0, df_res], axis=1)
    final_temp = pd.concat([temp_cur, final_temp], axis=0)
    
    # Plotting stars
    marg = 2
    if df_res.pval_1.to_numpy()[0] < 0.0167 and df_res.pval_2.to_numpy()[0] < 0.0167:
        row_ind = [i for i in range(len(inner_name_list)) if incon == inner_name_list[i]][0]
        ax[cou-1].text(row_ind-rshift, np.sum(vec[0])+marg, "*", fontsize=22)
        ax[cou].text(row_ind-rshift, np.sum(vec[1])+marg, "*", fontsize=22)
        
    return final_temp

# Original Count plot only

In [ ]:
# Redoing the count bargraph because I can not go back to it and make changes fast; it is too complex
final_temp_withincondi = pd.DataFrame()

final_temp_acrosscondi = pd.DataFrame()

df_long_tot = pd.DataFrame()

outer3_name = ['rot', 'trans']

bonfero_thresh = 0.0167
across_marg = 4

# estimator_type = 'sum'
estimator_type = 'mean' 

df_sig_cats = pd.DataFrame()

num_of_tests = 2



for exp in range(2):
    if exp == 0:
        exp_name = 'Rotation'
        anom = 'RO', 'PI', 'YA'
    else:
        exp_name = 'Translation'
        anom = 'LR', 'FB', 'UD'
        
    df = df_scalarmetics_exp[outer3_name[exp]]
    
    outer_name_list = ['sub', 'sup']
    inner_name_list = ['IC', 'EC', 'NC', 'NR']

    fig, ax = plt.subplots(2, 6, figsize=(20,10))
    po = 0
    cou = 0
    
    ax_all_vals_exp = []
    
    for i in range(3):  # 0=RO/LR, 1=PI/FB, 2=YA/UD
        
        ax_all_vals = []
        for j in range(1,3):  # 1=sub, 2=sup
            
            
            num_of_sub = len(df.subject.value_counts().to_numpy())
            print('num_of_sub: ', num_of_sub)
            
            # 1a) I need the counts per subject to get the population count for the error bars
            IC_vals = list(df.subject[(df.res_type == 1) & (df.ax == i) & (np.abs(df.ss) == j)].value_counts().to_numpy())
            
            # More than one or category requires that you sum the entries
            temp0_0 = list(df.subject[(df.res_type == 2) & (df.ax == i) & (np.abs(df.ss) == j)].value_counts().to_numpy())
            temp0_1 = list(df.subject[(df.res_type == 4) & (df.ax == i) & (np.abs(df.ss) == j)].value_counts().to_numpy())
            temp0_2 = list(df.subject[(df.res_type == 5) & (df.ax == i) & (np.abs(df.ss) == j)].value_counts().to_numpy())
            add1st = [x + y for x, y in zip_longest(temp0_0, temp0_1, fillvalue=0)]
            EC_vals = [x + y for x, y in zip_longest(add1st, temp0_2, fillvalue=0)]

            # More than one or category requires that you sum the entries
            temp1_0 = list(df.subject[(df.res_type == 3) & (df.ax == i) & (np.abs(df.ss) == j)].value_counts().to_numpy())
            temp1_1 = list(df.subject[(df.res_type == 6) & (df.ax == i) & (np.abs(df.ss) == j)].value_counts().to_numpy())
            temp1_2 = list(df.subject[(df.res_type == 7) & (df.ax == i) & (np.abs(df.ss) == j)].value_counts().to_numpy())
            add1st = [x + y for x, y in zip_longest(temp1_0, temp1_1, fillvalue=0)]
            NC_vals = [x + y for x, y in zip_longest(add1st, temp1_2, fillvalue=0)]

            NR_vals = df.subject[(df.res_type == 9) & (df.ax == i) & (np.abs(df.ss) == j)].value_counts().to_numpy()
            
            print('IC_vals : ', IC_vals)
            print('EC_vals : ', EC_vals)
            print('NC_vals : ', NC_vals)
            print('NR_vals : ', NR_vals)
            
            # Check for correctness : the sum of IC+EC+NC+NR == IC_vals+EC_vals+NC_vals+NR_vals
            sumofvals = np.sum(IC_vals) + np.sum(EC_vals) + np.sum(NC_vals) + np.sum(NR_vals)
            # print('sumofvals : ', sumofvals)

            # ----------------

            # 3) Normalize/scale the counts to values that it would be at if NO TRIALS were removed 
            
            # Each subject should have 42 trials : 
            # 3axis*2sub/sup*2direction (practice) +  3axis*2sub/sup*2direction (2times) + 6 sham = 12 + 24 + 6
            # 12+24=36/6 plotting categories(axis subORsup) = 6
            # so each axis subORsup should have 6 trials, not counting the 6 sham
            suspose2be_counts = num_of_sub*6 
            # print('suspose2be_counts : ', suspose2be_counts)

            multfactor = suspose2be_counts/sumofvals

            nor_IC_vals = [multfactor*i for i in IC_vals]
            nor_EC_vals = [multfactor*i for i in EC_vals]
            nor_NC_vals = [multfactor*i for i in NC_vals]
            nor_NR_vals = [multfactor*i for i in NR_vals]
            
            print('nor_IC_vals : ', nor_IC_vals)
            print('nor_EC_vals : ', nor_EC_vals)
            print('nor_NC_vals : ', nor_NC_vals)
            print('nor_NR_vals : ', nor_NR_vals)

            # ----------------

            # Make a new dataFrame with all the vals in one column and the label in the other put vals in a nested array
            all_vals = [nor_IC_vals, nor_EC_vals, nor_NC_vals, nor_NR_vals]
            longcol_num = []
            longcol_text = []
            for n1 in range(len(all_vals)):
                for n2 in range(len(all_vals[n1])):
                    longcol_num.append(all_vals[n1][n2])
                    longcol_text.append(inner_name_list[n1])

            col0 = pd.Series(np.ravel(longcol_text))
            col1 = pd.Series(np.ravel(longcol_num))
            temp = pd.concat([col0, col1], axis=1)
            df_long = temp.rename({0: 'str', 1: 'vals'}, axis=1)
            
            
            # Make an appended df_long across all conditions for statistical processing
            df_long_copy = temp.rename({0: 'str_%s_%s_%s' % (outer3_name[exp], anom[i], outer_name_list[j-1]), 1: 'vals_%s_%s_%s' % (outer3_name[exp], anom[i], outer_name_list[j-1])}, axis=1)
            df_long_tot = pd.concat([df_long_tot, df_long_copy], axis=1)
            # ----------------
            
            # Plot each subORsup per axis (for now)
            # sns.set(font_scale = 1.7, style="white", palette=None) 
            # OR
            sns.set(font_scale = 2) # default is without style and palette
            
            sns.color_palette("light:#90a4ae", as_cmap=True)  # Greys_d, light:#5A9
            
            
            if estimator_type == 'mean':
                # Plot the mean trial count
                if i == 0:
                    sns.barplot(x="str", y="vals", data=df_long, ax=ax[po,cou], ci="sd", capsize=.2, estimator=np.mean, palette="light:#90a4ae", errcolor="#3E26A8", linewidth=3, edgecolor="#3E26A8")
                elif i == 1:
                    sns.barplot(x="str", y="vals", data=df_long, ax=ax[po,cou], ci="sd", capsize=.2, estimator=np.mean, palette="light:#90a4ae", errcolor="#02BAC3", linewidth=3, edgecolor="#02BAC3")
                elif i == 2:
                    sns.barplot(x="str", y="vals", data=df_long, ax=ax[po,cou], ci="sd", capsize=.2, estimator=np.mean, palette="light:#90a4ae", errcolor="#F6EF1F", linewidth=3, edgecolor="#F6EF1F")
                ax[po,cou].set(ylim=(0, 25))
                ax[po,0].set_ylabel('Normalized mean trial count (%s)' % (exp_name), fontsize=22)
            elif estimator_type == 'sum':
                # Plot the sum trial count
                if i == 0:
                    sns.barplot(x="str", y="vals", data=df_long, ax=ax[po,cou], ci="sd", capsize=.2, estimator=np.sum, palette="light:#90a4ae", errcolor="#3E26A8", linewidth=3, edgecolor="#3E26A8")
                elif i == 1:
                    sns.barplot(x="str", y="vals", data=df_long, ax=ax[po,cou], ci="sd", capsize=.2, estimator=np.sum, palette="light:#90a4ae", errcolor="#02BAC3", linewidth=3, edgecolor="#02BAC3")
                elif i == 2:
                    sns.barplot(x="str", y="vals", data=df_long, ax=ax[po,cou], ci="sd", capsize=.2, estimator=np.sum, palette="light:#90a4ae", errcolor="#F6EF1F", linewidth=3, edgecolor="#F6EF1F")
                ax[po,cou].set(ylim=(0, 110))
                ax[po,0].set_ylabel('Normalized trial count (%s)' % (exp_name), fontsize=22)
            
            if cou > 0:
                ax[po,cou].set_ylabel(' ')
                ax[po,cou].set_yticks([])
            
            if j == 1:
                ax[po,cou].set_xlabel('%s sub' % (anom[i]), fontsize=22)
            elif j == 2:
                ax[po,cou].set_xlabel('%s sup' % (anom[i]), fontsize=22)
                
            ax_all_vals.append(all_vals)
            
            # ----------------
            # Forgot about this code : to plot only categories above the 95%CI
            # ----------------
            if cou == 1 or cou == 3 or cou == 5:
                ax_all_vals = np.array(ax_all_vals)
                print('length of ax_all_vals : ', ax_all_vals.shape)
                vec = make_a_properlist(np.concatenate((ax_all_vals[0], ax_all_vals[1]), axis=0))
                mean_dat, lower_tail, upper_tail = confidence_interval(vec, desired_CI=0.95)
                ax[po,cou-1].axhline(lower_tail, ls='--', linewidth=2, color='r')
                ax[po,cou].axhline(lower_tail, ls='--', linewidth=2, color='r')
                
            
                # Save which categories that are greater than the 95% lower CI for time response plotting
                
                # do not include NR: does not make sense to show response time for people who never 
                # responded (NR should all be around 15-16 seconds)
                for q in range(len(inner_name_list)-1):
                    for rr in range(2):
                        if np.sum(ax_all_vals[rr][q]) > lower_tail:
                            col4 = pd.Series(1) # number
                        else:
                            col4 = pd.Series(0) # number
                        col0 = pd.Series(outer3_name[exp])  # string
                        col1 = pd.Series(anom[i])  # string
                        col2 = pd.Series(outer_name_list[rr]) # string
                        col3 = pd.Series(inner_name_list[q])  # string
                        temp = pd.concat([col0, col1, col2, col3, col4], axis=1)
                        df_sig_cats = pd.concat([temp, df_sig_cats], axis=0)
            # ----------------           
            
            cou = cou + 1
            plt.xticks(fontsize=22)
            
    
    # ----------------
    # Statistics
    # ----------------
    # Trying a different way : because across conditions need all the data
    # 1) put needed data in a DataFrame, 2) read from DataFrame and do stats at the end:
    if exp == 0:
        # Within axes : differences with axis across same condition (Visually looked for any combinations) 
        which_anom = 'RO'
        incon = 'IC'
        print(exp, which_anom, incon)
        vec0 = df_long_tot.vals_rot_RO_sub[(df_long_tot.str_rot_RO_sub == incon)].to_numpy()
        print('vec0 : ', vec0)
        vec1 = df_long_tot.vals_rot_RO_sup[(df_long_tot.str_rot_RO_sup == incon)].to_numpy()
        print('vec1 : ', vec1)

        final_temp_withincondi = compare_within_condi(df_long_tot, incon, num_of_tests, final_temp_withincondi, bonfero_thresh, inner_name_list, across_marg, outer_name_list, anom, ax, po, exp, which_anom, estimator_type)
        incon = 'EC'
        print(exp, which_anom, incon)
        final_temp_withincondi = compare_within_condi(df_long_tot, incon, num_of_tests, final_temp_withincondi, bonfero_thresh, inner_name_list, across_marg, outer_name_list, anom, ax, po, exp, which_anom, estimator_type)
        incon = 'NC'
        print(exp, which_anom, incon)
        final_temp_withincondi = compare_within_condi(df_long_tot, incon, num_of_tests, final_temp_withincondi, bonfero_thresh, inner_name_list, across_marg, outer_name_list, anom, ax, po, exp, which_anom, estimator_type)

        which_anom = 'PI'
        incon = 'IC'
        print(exp, which_anom, incon)
        final_temp_withincondi = compare_within_condi(df_long_tot, incon, num_of_tests, final_temp_withincondi, bonfero_thresh, inner_name_list, across_marg, outer_name_list, anom, ax, po, exp, which_anom, estimator_type)
        incon = 'EC'
        print(exp, which_anom, incon)
        final_temp_withincondi = compare_within_condi(df_long_tot, incon, num_of_tests, final_temp_withincondi, bonfero_thresh, inner_name_list, across_marg, outer_name_list, anom, ax, po, exp, which_anom, estimator_type)
        incon = 'NC'
        print(exp, which_anom, incon)
        final_temp_withincondi = compare_within_condi(df_long_tot, incon, num_of_tests, final_temp_withincondi, bonfero_thresh, inner_name_list, across_marg, outer_name_list, anom, ax, po, exp, which_anom, estimator_type)

        which_anom = 'YA'
        incon = 'IC'
        print(exp, which_anom, incon)
        final_temp_withincondi = compare_within_condi(df_long_tot, incon, num_of_tests, final_temp_withincondi, bonfero_thresh, inner_name_list, across_marg, outer_name_list, anom, ax, po, exp, which_anom, estimator_type)
        incon = 'EC'
        print(exp, which_anom, incon)
        final_temp_withincondi = compare_within_condi(df_long_tot, incon, num_of_tests, final_temp_withincondi, bonfero_thresh, inner_name_list, across_marg, outer_name_list, anom, ax, po, exp, which_anom, estimator_type)
        incon = 'NC'
        print(exp, which_anom, incon)
        final_temp_withincondi = compare_within_condi(df_long_tot, incon, num_of_tests, final_temp_withincondi, bonfero_thresh, inner_name_list, across_marg, outer_name_list, anom, ax, po, exp, which_anom, estimator_type)

        # ---------

        # Across axes : differences between axis but same condition (Visually looked for any combinations)
        incon = 'IC'
        ss = 'sub'
        print(exp, ss, incon)
        final_temp_acrosscondi = compare_across_condi(df_long_tot, incon, num_of_tests, final_temp_acrosscondi, bonfero_thresh, inner_name_list, across_marg, outer_name_list, anom, ax, po, ss, exp, estimator_type)

        incon = 'EC'
        ss = 'sub'
        print(exp, ss, incon)
        final_temp_acrosscondi = compare_across_condi(df_long_tot, incon, num_of_tests, final_temp_acrosscondi, bonfero_thresh, inner_name_list, across_marg, outer_name_list, anom, ax, po, ss, exp, estimator_type)

        incon = 'NC'
        ss = 'sub'
        print(exp, ss, incon)
        final_temp_acrosscondi = compare_across_condi(df_long_tot, incon, num_of_tests, final_temp_acrosscondi, bonfero_thresh, inner_name_list, across_marg, outer_name_list, anom, ax, po, ss, exp, estimator_type)

        # ---------

        incon = 'IC'
        ss = 'sup'
        print(exp, ss, incon)
        final_temp_acrosscondi = compare_across_condi(df_long_tot, incon, num_of_tests, final_temp_acrosscondi, bonfero_thresh, inner_name_list, across_marg, outer_name_list, anom, ax, po, ss, exp, estimator_type)

        incon = 'EC'
        ss = 'sup'
        print(exp, ss, incon)
        final_temp_acrosscondi = compare_across_condi(df_long_tot, incon, num_of_tests, final_temp_acrosscondi, bonfero_thresh, inner_name_list, across_marg, outer_name_list, anom, ax, po, ss, exp, estimator_type)

        incon = 'NC'
        ss = 'sup'
        print(exp, ss, incon)
        final_temp_acrosscondi = compare_across_condi(df_long_tot, incon, num_of_tests, final_temp_acrosscondi, bonfero_thresh, inner_name_list, across_marg, outer_name_list, anom, ax, po, ss, exp, estimator_type)

    elif exp == 1:
        # Within axes : differences with axis across same condition (Visually looked for any combinations) 
        which_anom = 'LR'
        incon = 'IC'
        print(exp, which_anom, incon)
        final_temp_withincondi = compare_within_condi(df_long_tot, incon, num_of_tests, final_temp_withincondi, bonfero_thresh, inner_name_list, across_marg, outer_name_list, anom, ax, po, exp, which_anom, estimator_type)
        incon = 'EC'
        print(exp, which_anom, incon)
        final_temp_withincondi = compare_within_condi(df_long_tot, incon, num_of_tests, final_temp_withincondi, bonfero_thresh, inner_name_list, across_marg, outer_name_list, anom, ax, po, exp, which_anom, estimator_type)
        incon = 'NC'
        print(exp, which_anom, incon)
        final_temp_withincondi = compare_within_condi(df_long_tot, incon, num_of_tests, final_temp_withincondi, bonfero_thresh, inner_name_list, across_marg, outer_name_list, anom, ax, po, exp, which_anom, estimator_type)

        which_anom = 'FB'
        incon = 'IC'
        print(exp, which_anom, incon)
        final_temp_withincondi = compare_within_condi(df_long_tot, incon, num_of_tests, final_temp_withincondi, bonfero_thresh, inner_name_list, across_marg, outer_name_list, anom, ax, po, exp, which_anom, estimator_type)
        incon = 'EC'
        print(exp, which_anom, incon)
        final_temp_withincondi = compare_within_condi(df_long_tot, incon, num_of_tests, final_temp_withincondi, bonfero_thresh, inner_name_list, across_marg, outer_name_list, anom, ax, po, exp, which_anom, estimator_type)
        incon = 'NC'
        print(exp, which_anom, incon)
        final_temp_withincondi = compare_within_condi(df_long_tot, incon, num_of_tests, final_temp_withincondi, bonfero_thresh, inner_name_list, across_marg, outer_name_list, anom, ax, po, exp, which_anom, estimator_type)

        which_anom = 'UD'
        incon = 'IC'
        print(exp, which_anom, incon)
        final_temp_withincondi = compare_within_condi(df_long_tot, incon, num_of_tests, final_temp_withincondi, bonfero_thresh, inner_name_list, across_marg, outer_name_list, anom, ax, po, exp, which_anom, estimator_type)
        incon = 'EC'
        print(exp, which_anom, incon)
        final_temp_withincondi = compare_within_condi(df_long_tot, incon, num_of_tests, final_temp_withincondi, bonfero_thresh, inner_name_list, across_marg, outer_name_list, anom, ax, po, exp, which_anom, estimator_type)
        incon = 'NC'
        print(exp, which_anom, incon)
        final_temp_withincondi = compare_within_condi(df_long_tot, incon, num_of_tests, final_temp_withincondi, bonfero_thresh, inner_name_list, across_marg, outer_name_list, anom, ax, po, exp, which_anom, estimator_type)

        # ---------

        # Across axes : differences between axis but same condition (Visually looked for any combinations)
        incon = 'IC'
        ss = 'sub'
        print(exp, ss, incon)
        final_temp_acrosscondi = compare_across_condi(df_long_tot, incon, num_of_tests, final_temp_acrosscondi, bonfero_thresh, inner_name_list, across_marg, outer_name_list, anom, ax, po, ss, exp, estimator_type)

        incon = 'EC'
        ss = 'sub'
        print(exp, ss, incon)
        final_temp_acrosscondi = compare_across_condi(df_long_tot, incon, num_of_tests, final_temp_acrosscondi, bonfero_thresh, inner_name_list, across_marg, outer_name_list, anom, ax, po, ss, exp, estimator_type)

        incon = 'NC'
        ss = 'sub'
        print(exp, ss, incon)
        final_temp_acrosscondi = compare_across_condi(df_long_tot, incon, num_of_tests, final_temp_acrosscondi, bonfero_thresh, inner_name_list, across_marg, outer_name_list, anom, ax, po, ss, exp, estimator_type)

        # ---------

        incon = 'IC'
        ss = 'sup'
        print(exp, ss, incon)
        final_temp_acrosscondi = compare_across_condi(df_long_tot, incon, num_of_tests, final_temp_acrosscondi, bonfero_thresh, inner_name_list, across_marg, outer_name_list, anom, ax, po, ss, exp, estimator_type)

        incon = 'EC'
        ss = 'sup'
        print(exp, ss, incon)
        final_temp_acrosscondi = compare_across_condi(df_long_tot, incon, num_of_tests, final_temp_acrosscondi, bonfero_thresh, inner_name_list, across_marg, outer_name_list, anom, ax, po, ss, exp, estimator_type)

        incon = 'NC'
        ss = 'sup'
        print(exp, ss, incon)
        final_temp_acrosscondi = compare_across_condi(df_long_tot, incon, num_of_tests, final_temp_acrosscondi, bonfero_thresh, inner_name_list, across_marg, outer_name_list, anom, ax, po, ss, exp, estimator_type)

        plt.xticks(fontsize=22)
            
 
    plt.savefig('counts_%s.png' % (exp_name))
    

In [ ]:
incon = 'IC'
vec1 = df_long_tot.vals_rot_YA_sub[(df_long_tot.str_rot_YA_sub == incon)].to_numpy()
print('vec1 : ', vec1)
vec2 = df_long_tot.vals_rot_YA_sup[(df_long_tot.str_rot_YA_sup == incon)].to_numpy()
print('vec2 : ', vec2)
df_res = two_sample_stats(vec1, vec2, num_of_tests)
df_res
Dstat, pval_2 = stats.ranksums(vec1, vec2)
print('pval_2 : ', pval_2)

df_res

In [ ]:

# Pad vector with nan
if len(vec1) > len(vec2):
    longSIG = vec1
    shortSIG = vec2
    flag = 0
else: 
    shortSIG = vec1
    longSIG = vec2
    flag = 1

print('length of longSIG : ', len(longSIG))


# Reassign
if flag == 0:
    vec1 = longSIG
    vec2 = make_a_properlist(fix_shSIG)
elif flag == 1:
    vec1 = make_a_properlist(fix_shSIG)
    vec2 = longSIG

In [ ]:
df_long_tot

# Original TR plot only

In [ ]:
final_temp_withincondi = pd.DataFrame()
final_temp_acrosscondi = pd.DataFrame()

df_long_tot = pd.DataFrame()

outer3_name = ['rot', 'trans']

estimator_type = 'mean'
bonfero_thresh = 0.0167
across_marg = 4

num_of_tests = 2


for exp in range(2):
    if exp == 0:
        exp_name = 'Rotation'
        anom = 'RO', 'PI', 'YA'
    else:
        exp_name = 'Translation'
        anom = 'LR', 'FB', 'UD'
        
    df = df_scalarmetics_exp[outer3_name[exp]]
    
    outer_name_list = ['sub', 'sup']
    inner_name_list = ['IC', 'EC', 'NC', 'NR']

    fig, ax = plt.subplots(1, 6, figsize=(20,10))
    cou = 0
    
    ax_all_vals_exp = []
    for i in range(3):  # 0=RO/LR, 1=PI/FB, 2=YA/UD
        print('axis: ', i)
        
        ax_all_vals = []
        for j in range(1,3):  # 1=sub, 2=sup
            print('sub/sup: ', j)
            
            num_of_sub = len(df.subject.value_counts().to_numpy())
            # print('num_of_sub: ', num_of_sub)

            # 1a) I need the counts per subject to get the population count for the error bars
            IC_vals = list(df.TR[(df.res_type == 1) & (df.ax == i) & (np.abs(df.ss) == j)].to_numpy())
            
            # More than one or category requires that you sum the entries
            temp0_0 = list(df.TR[(df.res_type == 2) & (df.ax == i) & (np.abs(df.ss) == j)].to_numpy())
            print('res_type=2: ', temp0_0)
            temp0_1 = list(df.TR[(df.res_type == 4) & (df.ax == i) & (np.abs(df.ss) == j)].to_numpy())
            print('res_type=4: ', temp0_1)
            temp0_2 = list(df.TR[(df.res_type == 5) & (df.ax == i) & (np.abs(df.ss) == j)].to_numpy())
            print('res_type=5: ', temp0_2)
            
            applist = temp0_0, temp0_1, temp0_2
            EC_vals = make_a_properlist(applist)
            print('EC_vals: ', EC_vals)

            # More than one or category requires that you sum the entries
            temp1_0 = list(df.TR[(df.res_type == 3) & (df.ax == i) & (np.abs(df.ss) == j)].to_numpy())
            print('res_type=3: ', temp1_0)
            temp1_1 = list(df.TR[(df.res_type == 6) & (df.ax == i) & (np.abs(df.ss) == j)].to_numpy())
            print('res_type=6: ', temp1_1)
            temp1_2 = list(df.TR[(df.res_type == 7) & (df.ax == i) & (np.abs(df.ss) == j)].to_numpy())
            print('res_type=7: ', temp1_2)
            
            applist = temp1_0, temp1_1, temp1_2
            NC_vals = make_a_properlist(applist)
            print('NC_vals: ', NC_vals)
            
            # No Response is zero or nan because there was no response : do not even include it in the final plot
            # NR_vals = df.TR[(df.res_type == 9) & (df.ax == i) & (np.abs(df.ss) == j)].to_numpy()
            
            # Put desired values in a DataFrame
            # all_vals = [IC_vals, EC_vals, NC_vals, NR_vals]
            all_vals = [IC_vals, EC_vals, NC_vals]
            longcol_num = []
            longcol_text = []
            for n1 in range(len(all_vals)):
                for n2 in range(len(all_vals[n1])):
                    longcol_num.append(all_vals[n1][n2])
                    longcol_text.append(inner_name_list[n1])

            col0 = pd.Series(np.ravel(longcol_text))
            col1 = pd.Series(np.ravel(longcol_num))
            temp = pd.concat([col0, col1], axis=1)
            df_long = temp.rename({0: 'str', 1: 'vals'}, axis=1)
            
            # Make an appended df_long across all conditions for statistical processing
            df_long_copy = temp.rename({0: 'str_%s_%s_%s' % (outer3_name[exp], anom[i], outer_name_list[j-1]), 1: 'vals_%s_%s_%s' % (outer3_name[exp], anom[i], outer_name_list[j-1])}, axis=1)
            df_long_tot = pd.concat([df_long_tot, df_long_copy], axis=1)
            # ----------------
            
            # Plot each subORsup per axis (for now)
            # sns.set(font_scale = 1.7, style="white", palette=None) 
            # OR
            sns.set(font_scale = 2) # default is without style and palette
            
            sns.color_palette("light:#90a4ae", as_cmap=True)  # Greys_d, light:#5A9
            
            if i == 0:
                sns.barplot(x="str", y="vals", data=df_long, ax=ax[cou], ci="sd", capsize=.2, estimator=np.mean, palette="light:#90a4ae", errcolor="#3E26A8", linewidth=3, edgecolor="#3E26A8")
            elif i == 1:
                sns.barplot(x="str", y="vals", data=df_long, ax=ax[cou], ci="sd", capsize=.2, estimator=np.mean, palette="light:#90a4ae", errcolor="#02BAC3", linewidth=3, edgecolor="#02BAC3")
            elif i == 2:
                sns.barplot(x="str", y="vals", data=df_long, ax=ax[cou], ci="sd", capsize=.2, estimator=np.mean, palette="light:#90a4ae", errcolor="#F6EF1F", linewidth=3, edgecolor="#F6EF1F")
            ax[cou].set(ylim=(0, 35))
            
            # ax[0].set_ylabel('Normalized trial count (%s)' % (exp_name))
            ax[0].set_ylabel('Mean RT (%s)' % (exp_name), fontsize=22)
            
            plt.xticks(fontsize=22)
            
            if cou > 0:
                ax[cou].set_ylabel(' ')
                ax[cou].set_yticks([])
            
            if j == 1:
                ax[cou].set_xlabel('%s sub' % (anom[i]), fontsize=22)
            elif j == 2:
                ax[cou].set_xlabel('%s sup' % (anom[i]), fontsize=22)
                
            ax_all_vals.append(all_vals)
            
            cou = cou + 1
            
            
    # ----------------
    # Statistics
    # ----------------
    # Trying a different way : because across conditions need all the data
    # 1) put needed data in a DataFrame, 2) read from DataFrame and do stats at the end:
    if exp == 0:
        # Within axes : differences with axis across same condition (Visually looked for any combinations) 
        which_anom = 'RO'
        incon = 'EC'
        print(exp, which_anom, incon)
        final_temp_withincondi = compare_within_condi(df_long_tot, incon, num_of_tests, final_temp_withincondi, bonfero_thresh, inner_name_list, across_marg, outer_name_list, anom, ax,  exp, which_anom, estimator_type)
        incon = 'NC'
        print(exp, which_anom, incon)
        final_temp_withincondi = compare_within_condi(df_long_tot, incon, num_of_tests, final_temp_withincondi, bonfero_thresh, inner_name_list, across_marg, outer_name_list, anom, ax,  exp, which_anom, estimator_type)
        
        which_anom = 'PI'
        incon = 'IC'
        print(exp, which_anom, incon)
        final_temp_withincondi = compare_within_condi(df_long_tot, incon, num_of_tests, final_temp_withincondi, bonfero_thresh, inner_name_list, across_marg, outer_name_list, anom, ax,  exp, which_anom, estimator_type)
        
        which_anom = 'YA'
        incon = 'IC'
        print(exp, which_anom, incon)
        final_temp_withincondi = compare_within_condi(df_long_tot, incon, num_of_tests, final_temp_withincondi, bonfero_thresh, inner_name_list, across_marg, outer_name_list, anom, ax,  exp, which_anom, estimator_type)
        incon = 'EC'
        print(exp, which_anom, incon)
        final_temp_withincondi = compare_within_condi(df_long_tot, incon, num_of_tests, final_temp_withincondi, bonfero_thresh, inner_name_list, across_marg, outer_name_list, anom, ax,  exp, which_anom, estimator_type)
        incon = 'NC'
        print(exp, which_anom, incon)
        final_temp_withincondi = compare_within_condi(df_long_tot, incon, num_of_tests, final_temp_withincondi, bonfero_thresh, inner_name_list, across_marg, outer_name_list, anom, ax,  exp, which_anom, estimator_type)
        
        
        # Across axes : differences between axis but same condition (Visually looked for any combinations)
        incon = 'IC'
        ss = 'sub'
        print(exp, ss, incon)
        final_temp_acrosscondi = compare_across_condi(df_long_tot, incon, num_of_tests, final_temp_acrosscondi, bonfero_thresh, inner_name_list, across_marg, outer_name_list, anom, ax, ss, exp, estimator_type)
        
        incon = 'EC'
        ss = 'sub'
        print(exp, ss, incon)
        final_temp_acrosscondi = compare_across_condi(df_long_tot, incon, num_of_tests, final_temp_acrosscondi, bonfero_thresh, inner_name_list, across_marg, outer_name_list, anom, ax, ss, exp, estimator_type)
        
        incon = 'NC'
        ss = 'sub'
        print(exp, ss, incon)
        final_temp_acrosscondi = compare_across_condi(df_long_tot, incon, num_of_tests, final_temp_acrosscondi, bonfero_thresh, inner_name_list, across_marg, outer_name_list, anom, ax, ss, exp, estimator_type)
        
    elif exp == 1:
        # Within axes : differences with axis across same condition (Visually looked for any combinations) 
        which_anom = 'LR'
        incon = 'IC'
        print(exp, which_anom, incon)
        final_temp_withincondi = compare_within_condi(df_long_tot, incon, num_of_tests, final_temp_withincondi, bonfero_thresh, inner_name_list, across_marg, outer_name_list, anom, ax, exp, which_anom, estimator_type)
        incon = 'EC'
        print(exp, which_anom, incon)
        final_temp_withincondi = compare_within_condi(df_long_tot, incon, num_of_tests, final_temp_withincondi, bonfero_thresh, inner_name_list, across_marg, outer_name_list, anom, ax, exp, which_anom, estimator_type)
        incon = 'NC'
        print(exp, which_anom, incon)
        final_temp_withincondi = compare_within_condi(df_long_tot, incon, num_of_tests, final_temp_withincondi, bonfero_thresh, inner_name_list, across_marg, outer_name_list, anom, ax, exp, which_anom, estimator_type)
        
        which_anom = 'FB'
        incon = 'EC'
        print(exp, which_anom, incon)
        final_temp_withincondi = compare_within_condi(df_long_tot, incon, num_of_tests, final_temp_withincondi, bonfero_thresh, inner_name_list, across_marg, outer_name_list, anom, ax,  exp, which_anom, estimator_type)
        incon = 'NC'
        print(exp, which_anom, incon)
        final_temp_withincondi = compare_within_condi(df_long_tot, incon, num_of_tests, final_temp_withincondi, bonfero_thresh, inner_name_list, across_marg, outer_name_list, anom, ax,  exp, which_anom, estimator_type)
        
        which_anom = 'UD'
        incon = 'IC'
        print(exp, which_anom, incon)
        final_temp_withincondi = compare_within_condi(df_long_tot, incon, num_of_tests, final_temp_withincondi, bonfero_thresh, inner_name_list, across_marg, outer_name_list, anom, ax,  exp, which_anom, estimator_type)
        incon = 'EC'
        print(exp, which_anom, incon)
        final_temp_withincondi = compare_within_condi(df_long_tot, incon, num_of_tests, final_temp_withincondi, bonfero_thresh, inner_name_list, across_marg, outer_name_list, anom, ax,  exp, which_anom, estimator_type)
        incon = 'NC'
        print(exp, which_anom, incon)
        final_temp_withincondi = compare_within_condi(df_long_tot, incon, num_of_tests, final_temp_withincondi, bonfero_thresh, inner_name_list, across_marg, outer_name_list, anom, ax,  exp, which_anom, estimator_type)
        
        # Across axes : differences between axis but same condition (Visually looked for any combinations)
        incon = 'IC'
        ss = 'sub'
        print(exp, ss, incon)
        final_temp_acrosscondi = compare_across_condi(df_long_tot, incon, num_of_tests, final_temp_acrosscondi, bonfero_thresh, inner_name_list, across_marg, outer_name_list, anom, ax, ss, exp, estimator_type)
        
        incon = 'EC'
        ss = 'sub'
        print(exp, ss, incon)
        final_temp_acrosscondi = compare_across_condi(df_long_tot, incon, num_of_tests, final_temp_acrosscondi, bonfero_thresh, inner_name_list, across_marg, outer_name_list, anom, ax, ss, exp, estimator_type)
        
        incon = 'NC'
        ss = 'sub'
        print(exp, ss, incon)
        final_temp_acrosscondi = compare_across_condi(df_long_tot, incon, num_of_tests, final_temp_acrosscondi, bonfero_thresh, inner_name_list, across_marg, outer_name_list, anom, ax, ss, exp, estimator_type)
        
        incon = 'EC'
        ss = 'sup'
        print(exp, ss, incon)
        final_temp_acrosscondi = compare_across_condi(df_long_tot, incon, num_of_tests, final_temp_acrosscondi, bonfero_thresh, inner_name_list, across_marg, outer_name_list, anom, ax, ss, exp, estimator_type)
        
        incon = 'NC'
        ss = 'sup'
        print(exp, ss, incon)
        final_temp_acrosscondi = compare_across_condi(df_long_tot, incon, num_of_tests, final_temp_acrosscondi, bonfero_thresh, inner_name_list, across_marg, outer_name_list, anom, ax, ss, exp, estimator_type)
        
    plt.savefig('TR_%s.png' % (exp_name))

# Combined Count plot with TR plot
**Remember to run the cell twice, for the background of the plot and text to appear correctly.

In [ ]:
from subfunctions.plot_TR2subplot import *
from subfunctions.plot_count2subplot2 import *

dfexp_count_acrosscondi = pd.DataFrame()
dfexp_count_withincondi = pd.DataFrame()

dfexp_TR_acrosscondi = pd.DataFrame()
dfexp_TR_withincondi = pd.DataFrame()

eval_axis_dir = 'neg_ax'  # 'all_ax' = combine negative and positive axes, 'pos_ax' = positive axes only, 'neg_ax' = negative axes only

for exp in range(2):
    if exp == 0:
        exp_name = 'Rotation'
        anom = 'RO', 'PI', 'YA'
    else:
        exp_name = 'Translation'
        anom = 'LR', 'FB', 'UD'

    fig, ax = plt.subplots(3, 6, figsize=(20,10))
    po = 0
    cou = 0
    estimator_type = 'sum'
    %time cou, count_acrosscondi, count_withincondi, df_long_tot_count = plot_count2subplot2(po, df_scalarmetics_exp, cou, ax, plt, exp, exp_name, anom, estimator_type, eval_axis_dir)
    po = 1
    cou = 0
    estimator_type = 'mean'
    cou, count_acrosscondi, count_withincondi, df_long_tot_count = plot_count2subplot2(po, df_scalarmetics_exp, cou, ax, plt, exp, exp_name, anom, estimator_type, eval_axis_dir)
    po = 2
    cou = 0
    TR_acrosscondi, TR_withincondi, df_long_tot_TR = plot_TR2subplot(po, df_scalarmetics_exp, cou, ax, plt, exp, exp_name, anom, eval_axis_dir)
    
    # Stack both rot and trans into shared dataframes
    dfexp_count_acrosscondi = pd.concat([dfexp_count_acrosscondi, count_acrosscondi], axis=0)
    dfexp_count_withincondi = pd.concat([dfexp_count_withincondi, count_withincondi], axis=0)
    dfexp_TR_acrosscondi = pd.concat([dfexp_TR_acrosscondi, TR_acrosscondi], axis=0)
    dfexp_TR_withincondi = pd.concat([dfexp_TR_withincondi, TR_withincondi], axis=0)
    
    plt.savefig('counts_tr_%s_%s.png' % (exp_name, eval_axis_dir))
 

In [ ]:
# Plot for normalized count mean : this plot is what people are used to using 
# for evaluating error bars
from subfunctions.plot_count2subplot2 import *

dfexp_mean_count_acrosscondi = pd.DataFrame()
dfexp_mean_count_withincondi = pd.DataFrame()

estimator_type = 'mean'

eval_axis_dir = 'all_ax'  # 'all_ax' = combine negative and positive axes, 'all_ax' = positive axes only, 'neg_ax' = negative axes only

fig, ax = plt.subplots(2, 6, figsize=(20,10))
po = 0
cou = 0
for exp in range(2):
    if exp == 0:
        exp_name = 'Rotation'
        anom = 'RO', 'PI', 'YA'
    else:
        exp_name = 'Translation'
        anom = 'LR', 'FB', 'UD'

    cou, count_acrosscondi, count_withincondi, df_long_tot_mean_count = plot_count2subplot2(po, df_scalarmetics_exp, cou, ax, plt, exp, exp_name, anom, estimator_type, eval_axis_dir)
    po = po + 1
    cou = 0
    
    # Stack both rot and trans into shared dataframes
    dfexp_mean_count_acrosscondi = pd.concat([dfexp_mean_count_acrosscondi, count_acrosscondi], axis=0)
    dfexp_mean_count_withincondi = pd.concat([dfexp_mean_count_withincondi, count_withincondi], axis=0)
    
plt.savefig('meancounts_tr_RotTrans.png')

# Record count and TR statistics

In [ ]:
# Record count statistics for within condition (axis) : single star
dfexp_count_acrosscondi

In [ ]:
# double star
dfexp_count_withincondi

In [ ]:
dfexp_TR_acrosscondi

In [ ]:
dfexp_TR_withincondi

In [ ]:
dfexp_mean_count_acrosscondi

In [ ]:
dfexp_mean_count_withincondi

# Redoing Statistics so that they are more aligned with motion detection literature: group IC+EC and group NC+NR

## Compare across all axes for Rotation and Translation AND condensed Rotation and Translation figure

### Get count data

In [ ]:
def get_counts(df, df_long_tot, anom, outer3_name, outer_name_list):
    
    for i in range(3):
        ax_all_vals = []
        for j in range(1,3):  # 1=sub, 2=sup

            num_of_sub = len(df.subject.value_counts().to_numpy())
            print('num_of_sub: ', num_of_sub)

            # 1a) I need the counts per subject to get the population count for the error bars
            IC_vals = list(df.subject[(df.res_type == 1) & (df.ax == i) & (np.abs(df.ss) == j)].value_counts().to_numpy())

            # More than one or category requires that you sum the entries
            temp0_0 = list(df.subject[(df.res_type == 2) & (df.ax == i) & (np.abs(df.ss) == j)].value_counts().to_numpy())
            temp0_1 = list(df.subject[(df.res_type == 4) & (df.ax == i) & (np.abs(df.ss) == j)].value_counts().to_numpy())
            temp0_2 = list(df.subject[(df.res_type == 5) & (df.ax == i) & (np.abs(df.ss) == j)].value_counts().to_numpy())
            add1st = [x + y for x, y in zip_longest(temp0_0, temp0_1, fillvalue=0)]
            EC_vals = [x + y for x, y in zip_longest(add1st, temp0_2, fillvalue=0)]

            # More than one or category requires that you sum the entries
            temp1_0 = list(df.subject[(df.res_type == 3) & (df.ax == i) & (np.abs(df.ss) == j)].value_counts().to_numpy())
            temp1_1 = list(df.subject[(df.res_type == 6) & (df.ax == i) & (np.abs(df.ss) == j)].value_counts().to_numpy())
            temp1_2 = list(df.subject[(df.res_type == 7) & (df.ax == i) & (np.abs(df.ss) == j)].value_counts().to_numpy())
            add1st = [x + y for x, y in zip_longest(temp1_0, temp1_1, fillvalue=0)]
            NC_vals = [x + y for x, y in zip_longest(add1st, temp1_2, fillvalue=0)]

            NR_vals = df.subject[(df.res_type == 9) & (df.ax == i) & (np.abs(df.ss) == j)].value_counts().to_numpy()

            print('IC_vals : ', IC_vals)
            print('EC_vals : ', EC_vals)
            print('NC_vals : ', NC_vals)
            print('NR_vals : ', NR_vals)

            # Check for correctness : the sum of IC+EC+NC+NR == IC_vals+EC_vals+NC_vals+NR_vals
            sumofvals = np.sum(IC_vals) + np.sum(EC_vals) + np.sum(NC_vals) + np.sum(NR_vals)
            # print('sumofvals : ', sumofvals)

            # ----------------

            # 3) Normalize/scale the counts to values that it would be at if NO TRIALS were removed 

            # Each subject should have 42 trials : 
            # 3axis*2sub/sup*2direction (practice) +  3axis*2sub/sup*2direction (2times) + 6 sham = 12 + 24 + 6
            # 12+24=36/6 plotting categories(axis subORsup) = 6
            # so each axis subORsup should have 6 trials, not counting the 6 sham
            suspose2be_counts = num_of_sub*6 
            # print('suspose2be_counts : ', suspose2be_counts)

            multfactor = suspose2be_counts/sumofvals

            nor_IC_vals = [multfactor*i for i in IC_vals]
            nor_EC_vals = [multfactor*i for i in EC_vals]
            nor_NC_vals = [multfactor*i for i in NC_vals]
            nor_NR_vals = [multfactor*i for i in NR_vals]

            print('nor_IC_vals : ', nor_IC_vals)
            print('nor_EC_vals : ', nor_EC_vals)
            print('nor_NC_vals : ', nor_NC_vals)
            print('nor_NR_vals : ', nor_NR_vals)

            # ----------------

            # Make a new dataFrame with all the vals in one column and the label in the other put vals in a nested array
            all_vals = [nor_IC_vals, nor_EC_vals, nor_NC_vals, nor_NR_vals]
            print('all_vals : ', all_vals)
            
            longcol_num = []
            longcol_text = []
            for n1 in range(len(all_vals)):
                for n2 in range(len(all_vals[n1])):
                    longcol_num.append(all_vals[n1][n2])
                    longcol_text.append(inner_name_list[n1])
            
            
            col0 = pd.Series(np.ravel(longcol_text))
            col1 = pd.Series(np.ravel(longcol_num))
            
            #print('col0 : ', col0)
            #print('col1 : ', col1)
            
            temp = pd.concat([col0, col1], axis=1)
            #df_long = temp.rename({0: 'str', 1: 'vals'}, axis=1)


            # Make an appended df_long across all conditions for statistical processing
            df_long_copy = temp.rename({0: 'str_%s_%s_%s' % (outer3_name[exp], anom[i], outer_name_list[j-1]), 1: 'vals_%s_%s_%s' % (outer3_name[exp], anom[i], outer_name_list[j-1])}, axis=1)
            df_long_tot = pd.concat([df_long_tot, df_long_copy], axis=1)
            # ----------------

    return df_long_tot

In [ ]:
def get_TR(df, df_long_tot, anom, outer3_name, outer_name_list):
    
    for i in range(3):
        ax_all_vals = []
        for j in range(1,3):  # 1=sub, 2=sup

            num_of_sub = len(df.subject.value_counts().to_numpy())
            print('num_of_sub: ', num_of_sub)
            
            # ------------------------------------
            # Combined axis direction evaluation 
            # ------------------------------------
            # 1a) I need the counts per subject to get the population count for the error bars
            IC_vals = list(df.TR[(df.res_type == 1) & (df.ax == i) & (np.abs(df.ss) == j)].to_numpy())
                
            # More than one or category requires that you sum the entries
            temp0_0 = list(df.TR[(df.res_type == 2) & (df.ax == i) & (np.abs(df.ss) == j)].to_numpy())
            print('res_type=2: ', temp0_0)
            temp0_1 = list(df.TR[(df.res_type == 4) & (df.ax == i) & (np.abs(df.ss) == j)].to_numpy())
            print('res_type=4: ', temp0_1)
            temp0_2 = list(df.TR[(df.res_type == 5) & (df.ax == i) & (np.abs(df.ss) == j)].to_numpy())
            print('res_type=5: ', temp0_2)

            applist = temp0_0, temp0_1, temp0_2
            EC_vals = make_a_properlist(applist)
            print('EC_vals: ', EC_vals)

            # More than one or category requires that you sum the entries
            temp1_0 = list(df.TR[(df.res_type == 3) & (df.ax == i) & (np.abs(df.ss) == j)].to_numpy())
            print('res_type=3: ', temp1_0)
            temp1_1 = list(df.TR[(df.res_type == 6) & (df.ax == i) & (np.abs(df.ss) == j)].to_numpy())
            print('res_type=6: ', temp1_1)
            temp1_2 = list(df.TR[(df.res_type == 7) & (df.ax == i) & (np.abs(df.ss) == j)].to_numpy())
            print('res_type=7: ', temp1_2)

            applist = temp1_0, temp1_1, temp1_2
            NC_vals = make_a_properlist(applist)
            print('NC_vals: ', NC_vals)
            
            # Put desired values in a DataFrame
            # all_vals = [IC_vals, EC_vals, NC_vals, NR_vals]
            all_vals = [IC_vals, EC_vals, NC_vals]
            longcol_num = []
            longcol_text = []
            for n1 in range(len(all_vals)):
                for n2 in range(len(all_vals[n1])):
                    longcol_num.append(all_vals[n1][n2])
                    longcol_text.append(inner_name_list[n1])

            col0 = pd.Series(np.ravel(longcol_text))
            col1 = pd.Series(np.ravel(longcol_num))
            
            temp = pd.concat([col0, col1], axis=1)
            
            # Make an appended df_long across all conditions for statistical processing
            df_long_copy = temp.rename({0: 'str_%s_%s_%s' % (outer3_name[exp], anom[i], outer_name_list[j-1]), 1: 'vals_%s_%s_%s' % (outer3_name[exp], anom[i], outer_name_list[j-1])}, axis=1)
            df_long_tot = pd.concat([df_long_tot, df_long_copy], axis=1)
            # ----------------

    return df_long_tot

In [ ]:
df_long_tot = pd.DataFrame()

df_long_totTR = pd.DataFrame()

outer3_name = ['rot', 'trans']
outer_name_list = ['sub', 'sup']
inner_name_list = ['IC', 'EC', 'NC', 'NR']


exp = 0
exp_name = 'Rotation'
anom = 'RO', 'PI', 'YA'
df = df_scalarmetics_exp[outer3_name[exp]]

# counts
df_long_tot = get_counts(df, df_long_tot, anom, outer3_name, outer_name_list)

# TR
df_long_totTR = get_TR(df, df_long_totTR, anom, outer3_name, outer_name_list)
# -----------------------------------
    
exp = 1
exp_name = 'Translation'
anom = 'LR', 'FB', 'UD'
df = df_scalarmetics_exp[outer3_name[exp]]

# counts
df_long_tot = get_counts(df, df_long_tot, anom, outer3_name, outer_name_list)

# TR
df_long_totTR = get_TR(df, df_long_totTR, anom, outer3_name, outer_name_list)

# Rotation has 4 more participants than Translation : We compare the two without adjusting 
# Translation because we compare the distribution of the data

In [ ]:
df_long_tot

### Statistics of count data

In [ ]:
# Statistics count : comparing speed categories within axis : using IC+EC sub vs IC+EC sup
num_of_tests = 2
final_temp_acrosscondi_subsup = pd.DataFrame()

inc1 = 'IC'
inc2 = 'EC'

incon_list = ['IC+EC', 'NC+NR']

nom = ['RO', 'PI', 'YA', 'LR', 'FB', 'UD']

for incon in incon_list:
    for inom in nom:

        if inom == 'RO':
            vecA = df_long_tot.vals_rot_RO_sub[(df_long_tot.str_rot_RO_sub == inc1) | (df_long_tot.str_rot_RO_sub == inc2)].to_numpy()
            vecB = df_long_tot.vals_rot_RO_sup[(df_long_tot.str_rot_RO_sup == inc1) | (df_long_tot.str_rot_RO_sup == inc2)].to_numpy()
        elif inom == 'PI':
            vecA = df_long_tot.vals_rot_PI_sub[(df_long_tot.str_rot_PI_sub == inc1) | (df_long_tot.str_rot_PI_sub == inc2)].to_numpy()
            vecB = df_long_tot.vals_rot_PI_sup[(df_long_tot.str_rot_PI_sup == inc1) | (df_long_tot.str_rot_PI_sup == inc2)].to_numpy()
        elif inom == 'YA':
            vecA = df_long_tot.vals_rot_YA_sub[(df_long_tot.str_rot_YA_sub == inc1) | (df_long_tot.str_rot_YA_sub == inc2)].to_numpy()
            vecB = df_long_tot.vals_rot_YA_sup[(df_long_tot.str_rot_YA_sup == inc1) | (df_long_tot.str_rot_YA_sup == inc2)].to_numpy()
        elif inom == 'LR':
            vecA = df_long_tot.vals_trans_LR_sub[(df_long_tot.str_trans_LR_sub == inc1) | (df_long_tot.str_trans_LR_sub == inc2)].to_numpy()
            vecB = df_long_tot.vals_trans_LR_sup[(df_long_tot.str_trans_LR_sup == inc1) | (df_long_tot.str_trans_LR_sup == inc2)].to_numpy()
        elif inom == 'FB':
            vecA = df_long_tot.vals_trans_FB_sub[(df_long_tot.str_trans_FB_sub == inc1) | (df_long_tot.str_trans_FB_sub == inc2)].to_numpy()
            vecB = df_long_tot.vals_trans_FB_sup[(df_long_tot.str_trans_FB_sup == inc1) | (df_long_tot.str_trans_FB_sup == inc2)].to_numpy()
        elif inom == 'UD':
            vecA = df_long_tot.vals_trans_UD_sub[(df_long_tot.str_trans_UD_sub == inc1) | (df_long_tot.str_trans_UD_sub == inc2)].to_numpy()
            vecB = df_long_tot.vals_trans_UD_sup[(df_long_tot.str_trans_UD_sup == inc1) | (df_long_tot.str_trans_UD_sup == inc2)].to_numpy()

        # Statistical tests
        df_res = two_sample_stats(vecA, vecB, num_of_tests)
        compare_name = '%s %s sub sup' % (incon, inom)
        # Saving stats in a DataFrame
        col0 = pd.Series('%s' % (compare_name))  # string
        temp_cur = pd.concat([col0, df_res], axis=1)
        final_temp_acrosscondi_subsup = pd.concat([temp_cur, final_temp_acrosscondi_subsup], axis=0)


In [ ]:
final_temp_acrosscondi_subsup

In [ ]:
# Statistics count : comparing across axes with respect to same speed category

nom = ['RO', 'PI', 'YA', 'LR', 'FB', 'UD']

num_of_tests = 2
final_temp_acrosscondi = pd.DataFrame()

incon_list = ['IC+EC', 'NC+NR']

for incon in incon_list:
    if incon == 'IC+EC':
        inc1 = 'IC'
        inc2 = 'EC'
    elif incon == 'NC+NR':
        inc1 = 'NC'
        inc2 = 'NR'
    
    for ss in outer_name_list:
        if ss == 'sub':
            vec0 = df_long_tot.vals_rot_RO_sub[(df_long_tot.str_rot_RO_sub == inc1) | (df_long_tot.str_rot_RO_sub == inc2)].to_numpy()
            vec1 = df_long_tot.vals_rot_PI_sub[(df_long_tot.str_rot_PI_sub == inc1) | (df_long_tot.str_rot_PI_sub == inc2)].to_numpy()
            vec2 = df_long_tot.vals_rot_YA_sub[(df_long_tot.str_rot_YA_sub == inc1) | (df_long_tot.str_rot_YA_sub == inc2)].to_numpy()
            vec3 = df_long_tot.vals_trans_LR_sub[(df_long_tot.str_trans_LR_sub == inc1) | (df_long_tot.str_trans_LR_sub == inc2)].to_numpy()
            vec4 = df_long_tot.vals_trans_FB_sub[(df_long_tot.str_trans_FB_sub == inc1) | (df_long_tot.str_trans_FB_sub == inc2)].to_numpy()
            vec5 = df_long_tot.vals_trans_UD_sub[(df_long_tot.str_trans_UD_sub == inc1) | (df_long_tot.str_trans_UD_sub == inc2)].to_numpy()  
        elif ss == 'sup':
            vec0 = df_long_tot.vals_rot_RO_sup[(df_long_tot.str_rot_RO_sup == inc1) | (df_long_tot.str_rot_RO_sup == inc2)].to_numpy()
            vec1 = df_long_tot.vals_rot_PI_sup[(df_long_tot.str_rot_PI_sup == inc1) | (df_long_tot.str_rot_PI_sup == inc2)].to_numpy()
            vec2 = df_long_tot.vals_rot_YA_sup[(df_long_tot.str_rot_YA_sup == inc1) | (df_long_tot.str_rot_YA_sup == inc2)].to_numpy()
            vec3 = df_long_tot.vals_trans_LR_sup[(df_long_tot.str_trans_LR_sup == inc1) | (df_long_tot.str_trans_LR_sup == inc2)].to_numpy()
            vec4 = df_long_tot.vals_trans_FB_sup[(df_long_tot.str_trans_FB_sup == inc1) | (df_long_tot.str_trans_FB_sup == inc2)].to_numpy()
            vec5 = df_long_tot.vals_trans_UD_sup[(df_long_tot.str_trans_UD_sup == inc1) | (df_long_tot.str_trans_UD_sup == inc2)].to_numpy()
            
        vec_comp = [vec0, vec1, vec2, vec3, vec4, vec5]
        
        
        # Statistics 
        lister = list(range(len(vec_comp)-1, 0, -1))
        # print('lister : ', lister)
        acc4res = len(lister)-1      # account for resets
        tot = np.sum(lister)
        # print('tot : ', tot)

        frt = 0
        sec = frt+1
        c = 0
        for i in range(tot+acc4res):
            print('i : ', i)
            if c < lister[frt]:

                vecA = vec_comp[frt]
                vecB = vec_comp[sec]
                print('vecA : ', vecA)
                print('vecB : ', vecB)

                # Statistical tests
                df_res = two_sample_stats(vecA, vecB, num_of_tests)
                compare_name = '%s %s %s %s' % (incon, ss, nom[frt], nom[sec])
                # Saving stats in a DataFrame
                col0 = pd.Series('%s' % (compare_name))  # string
                temp_cur = pd.concat([col0, df_res], axis=1)
                final_temp_acrosscondi = pd.concat([temp_cur, final_temp_acrosscondi], axis=0)

                sec = sec + 1
                c = c + 1
            elif c >= lister[frt]:
                # reset
                c = 0
                frt = frt+1
                sec = frt+1

In [ ]:
# Statistics RT : comparing correct to not-correct

nom = ['RO', 'PI', 'YA', 'LR', 'FB', 'UD']

num_of_tests = 2
final_temp_acrosscondiTR = pd.DataFrame()

incon_list = ['IC+EC', 'NC+NR']

for incon in incon_list:
    if incon == 'IC+EC':
        inc1 = 'IC'
        inc2 = 'EC'
    elif incon == 'NC+NR':
        inc1 = 'NC'
        inc2 = 'NR'
    
    for ss in outer_name_list:
        if ss == 'sub':
            vec0 = df_long_totTR.vals_rot_RO_sub[(df_long_totTR.str_rot_RO_sub == inc1) | (df_long_totTR.str_rot_RO_sub == inc2)].to_numpy()
            vec1 = df_long_totTR.vals_rot_PI_sub[(df_long_totTR.str_rot_PI_sub == inc1) | (df_long_totTR.str_rot_PI_sub == inc2)].to_numpy()
            vec2 = df_long_totTR.vals_rot_YA_sub[(df_long_totTR.str_rot_YA_sub == inc1) | (df_long_totTR.str_rot_YA_sub == inc2)].to_numpy()
            vec3 = df_long_totTR.vals_trans_LR_sub[(df_long_totTR.str_trans_LR_sub == inc1) | (df_long_totTR.str_trans_LR_sub == inc2)].to_numpy()
            vec4 = df_long_totTR.vals_trans_FB_sub[(df_long_totTR.str_trans_FB_sub == inc1) | (df_long_totTR.str_trans_FB_sub == inc2)].to_numpy()
            vec5 = df_long_totTR.vals_trans_UD_sub[(df_long_totTR.str_trans_UD_sub == inc1) | (df_long_totTR.str_trans_UD_sub == inc2)].to_numpy()  
        elif ss == 'sup':
            vec0 = df_long_totTR.vals_rot_RO_sup[(df_long_totTR.str_rot_RO_sup == inc1) | (df_long_totTR.str_rot_RO_sup == inc2)].to_numpy()
            vec1 = df_long_totTR.vals_rot_PI_sup[(df_long_totTR.str_rot_PI_sup == inc1) | (df_long_totTR.str_rot_PI_sup == inc2)].to_numpy()
            vec2 = df_long_totTR.vals_rot_YA_sup[(df_long_totTR.str_rot_YA_sup == inc1) | (df_long_totTR.str_rot_YA_sup == inc2)].to_numpy()
            vec3 = df_long_totTR.vals_trans_LR_sup[(df_long_totTR.str_trans_LR_sup == inc1) | (df_long_totTR.str_trans_LR_sup == inc2)].to_numpy()
            vec4 = df_long_totTR.vals_trans_FB_sup[(df_long_totTR.str_trans_FB_sup == inc1) | (df_long_totTR.str_trans_FB_sup == inc2)].to_numpy()
            vec5 = df_long_totTR.vals_trans_UD_sup[(df_long_totTR.str_trans_UD_sup == inc1) | (df_long_totTR.str_trans_UD_sup == inc2)].to_numpy()
            
        vec_comp = [vec0, vec1, vec2, vec3, vec4, vec5]
        
        
        # Statistics 
        lister = list(range(len(vec_comp)-1, 0, -1))
        # print('lister : ', lister)
        acc4res = len(lister)-1      # account for resets
        tot = np.sum(lister)
        # print('tot : ', tot)

        frt = 0
        sec = frt+1
        c = 0
        for i in range(tot+acc4res):
            print('i : ', i)
            if c < lister[frt]:

                vecA = vec_comp[frt]
                vecB = vec_comp[sec]
                print('vecA : ', vecA)
                print('vecB : ', vecB)

                # Statistical tests
                df_res = two_sample_stats(vecA, vecB, num_of_tests)
                compare_name = '%s %s %s %s' % (incon, ss, nom[frt], nom[sec])
                # Saving stats in a DataFrame
                col0 = pd.Series('%s' % (compare_name))  # string
                temp_cur = pd.concat([col0, df_res], axis=1)
                final_temp_acrosscondiTR = pd.concat([temp_cur, final_temp_acrosscondiTR], axis=0)

                sec = sec + 1
                c = c + 1
            elif c >= lister[frt]:
                # reset
                c = 0
                frt = frt+1
                sec = frt+1

In [ ]:
final_temp_acrosscondi

In [ ]:
final_temp_acrosscondiTR

In [ ]:
# We only need rows where pval_1 and pval_2 are less than 0.0167 
final_temp_acrosscondi[(final_temp_acrosscondi.pval_1 < 0.0167) & (final_temp_acrosscondi.pval_2 < 0.0167)]

In [ ]:
# We only need rows where pval_1 and pval_2 are less than 0.0167 
final_temp_acrosscondiTR[(final_temp_acrosscondiTR.pval_1 < 0.0167) & (final_temp_acrosscondiTR.pval_2 < 0.0167)]

### Plotting count data

In [ ]:
def repeating_stuff(vec0_vals, vec1_vals, vec2_vals, vec3_vals, vec4_vals, vec5_vals, incon):
    
    vec0_str0 = 'RO ' * len(vec0_vals)
    vec0_str = vec0_str0.split()

    vec1_str0 = 'PI ' * len(vec1_vals)
    vec1_str = vec1_str0.split()

    vec2_str0 = 'YA ' * len(vec2_vals)
    vec2_str = vec2_str0.split()

    vec3_str0 = 'LR ' * len(vec3_vals)
    vec3_str = vec3_str0.split()

    vec4_str0 = 'FB ' * len(vec4_vals)
    vec4_str = vec4_str0.split()

    vec5_str0 = 'UD ' * len(vec5_vals)
    vec5_str = vec5_str0.split()

    # Plotting combined rotational and translation data
    vec_comp_vals = pd.concat([vec0_vals, vec1_vals, vec2_vals, vec3_vals, vec4_vals, vec5_vals])
    vec_comp_vals.reset_index(drop=True, inplace=True)

    data = make_a_properlist([vec0_str, vec1_str, vec2_str, vec3_str, vec4_str, vec5_str])
    vec_comp_str = pd.Series(data)
    vec_comp_str.reset_index(drop=True, inplace=True)

    a = incon+" "
    data_res_type = a * len(vec_comp_str)
    data_res_type = data_res_type.split()
    vec_comp_res_type = pd.Series(data_res_type)
    vec_comp_res_type.reset_index(drop=True, inplace=True)

    temp = pd.concat([vec_comp_str, vec_comp_vals, vec_comp_res_type], axis=1)
    
    return temp

In [ ]:
# Plot of half the data
# incon_list = ['IC', 'EC', 'NC', 'NR']
incon_list = ['IC', 'NC']

for ss in outer_name_list:
    if ss == 'sub':
        
        condensed_df_sub = pd.DataFrame()

        for incon in incon_list:
            vec0_vals = df_long_tot.vals_rot_RO_sub[(df_long_tot.str_rot_RO_sub == incon)]
            vec1_vals = df_long_tot.vals_rot_PI_sub[(df_long_tot.str_rot_PI_sub == incon)]
            vec2_vals = df_long_tot.vals_rot_YA_sub[(df_long_tot.str_rot_YA_sub == incon)]
            vec3_vals = df_long_tot.vals_trans_LR_sub[(df_long_tot.str_trans_LR_sub == incon)]
            vec4_vals = df_long_tot.vals_trans_FB_sub[(df_long_tot.str_trans_FB_sub == incon)]
            vec5_vals = df_long_tot.vals_trans_UD_sub[(df_long_tot.str_trans_UD_sub == incon)]
            
            temp = repeating_stuff(vec0_vals, vec1_vals, vec2_vals, vec3_vals, vec4_vals, vec5_vals, incon)
            
            temp = temp.rename({0: 'str', 1: 'vals', 2: 'res_type'}, axis=1)
            
            #condensed_df_sub = pd.concat([condensed_df_sub, temp], axis=0)
            # OR
            condensed_df_sub = condensed_df_sub.append(temp)
            
            print('shape of condensed_df_sub: ', condensed_df_sub.shape)
            
            condensed_df_sub.reset_index(drop=True, inplace=True)
            condensed_df_sub = condensed_df_sub.rename({0: 'str', 1: 'vals', 2: 'res_type'}, axis=1)
            
                
    elif ss == 'sup':
            
        condensed_df_sup = pd.DataFrame()
            
        for incon in incon_list:
            vec0_vals = df_long_tot.vals_rot_RO_sup[(df_long_tot.str_rot_RO_sup == incon)]
            vec1_vals = df_long_tot.vals_rot_PI_sup[(df_long_tot.str_rot_PI_sup == incon)]
            vec2_vals = df_long_tot.vals_rot_YA_sup[(df_long_tot.str_rot_YA_sup == incon)]
            vec3_vals = df_long_tot.vals_trans_LR_sup[(df_long_tot.str_trans_LR_sup == incon)]
            vec4_vals = df_long_tot.vals_trans_FB_sup[(df_long_tot.str_trans_FB_sup == incon)]
            vec5_vals = df_long_tot.vals_trans_UD_sup[(df_long_tot.str_trans_UD_sup == incon)]

            temp = repeating_stuff(vec0_vals, vec1_vals, vec2_vals, vec3_vals, vec4_vals, vec5_vals, incon)
            temp = temp.rename({0: 'str', 1: 'vals', 2: 'res_type'}, axis=1)
            
            #condensed_df_sup = pd.concat([condensed_df_sup, temp], axis=0)
            # OR
            condensed_df_sup = condensed_df_sup.append(temp)
            
            
            condensed_df_sup.reset_index(drop=True, inplace=True)
            condensed_df_sup = condensed_df_sup.rename({0: 'str', 1: 'vals', 2: 'res_type'}, axis=1)

In [ ]:
# Plot
incon_list = ['IC+EC', 'NC+NR']

for ss in outer_name_list:
    if ss == 'sub':
        
        condensed_df_sub_2cat = pd.DataFrame()

        for incon in incon_list:
            if incon == 'IC+EC':
                inc1 = 'IC'
                inc2 = 'EC'
            elif incon == 'NC+NR':
                inc1 = 'NC'
                inc2 = 'NR'

            vec0_vals = df_long_tot.vals_rot_RO_sub[(df_long_tot.str_rot_RO_sub == inc1) | (df_long_tot.str_rot_RO_sub == inc2)]
            vec1_vals = df_long_tot.vals_rot_PI_sub[(df_long_tot.str_rot_PI_sub == inc1) | (df_long_tot.str_rot_PI_sub == inc2)]
            vec2_vals = df_long_tot.vals_rot_YA_sub[(df_long_tot.str_rot_YA_sub == inc1) | (df_long_tot.str_rot_YA_sub == inc2)]
            vec3_vals = df_long_tot.vals_trans_LR_sub[(df_long_tot.str_trans_LR_sub == inc1) | (df_long_tot.str_trans_LR_sub == inc2)]
            vec4_vals = df_long_tot.vals_trans_FB_sub[(df_long_tot.str_trans_FB_sub == inc1) | (df_long_tot.str_trans_FB_sub == inc2)]
            vec5_vals = df_long_tot.vals_trans_UD_sub[(df_long_tot.str_trans_UD_sub == inc1) | (df_long_tot.str_trans_UD_sub == inc2)]
            
            temp = repeating_stuff(vec0_vals, vec1_vals, vec2_vals, vec3_vals, vec4_vals, vec5_vals, incon)
            
            temp = temp.rename({0: 'str', 1: 'vals', 2: 'res_type'}, axis=1)
            
            #condensed_df_sub_2cat = pd.concat([condensed_df_sub_2cat, temp], axis=0)
            # OR
            condensed_df_sub_2cat = condensed_df_sub_2cat.append(temp)
            
            print('shape of condensed_df_sub_2cat: ', condensed_df_sub_2cat.shape)
            
            condensed_df_sub_2cat.reset_index(drop=True, inplace=True)
            condensed_df_sub_2cat = condensed_df_sub_2cat.rename({0: 'str', 1: 'vals', 2: 'res_type'}, axis=1)
            
                
    elif ss == 'sup':
            
        condensed_df_sup_2cat = pd.DataFrame()
            
        for incon in incon_list:
            if incon == 'IC+EC':
                inc1 = 'IC'
                inc2 = 'EC'
            elif incon == 'NC+NR':
                inc1 = 'NC'
                inc2 = 'NR'

            vec0_vals = df_long_tot.vals_rot_RO_sup[(df_long_tot.str_rot_RO_sup == inc1) | (df_long_tot.str_rot_RO_sup == inc2)]
            vec1_vals = df_long_tot.vals_rot_PI_sup[(df_long_tot.str_rot_PI_sup == inc1) | (df_long_tot.str_rot_PI_sup == inc2)]
            vec2_vals = df_long_tot.vals_rot_YA_sup[(df_long_tot.str_rot_YA_sup == inc1) | (df_long_tot.str_rot_YA_sup == inc2)]
            vec3_vals = df_long_tot.vals_trans_LR_sup[(df_long_tot.str_trans_LR_sup == inc1) | (df_long_tot.str_trans_LR_sup == inc2)]
            vec4_vals = df_long_tot.vals_trans_FB_sup[(df_long_tot.str_trans_FB_sup == inc1) | (df_long_tot.str_trans_FB_sup == inc2)]
            vec5_vals = df_long_tot.vals_trans_UD_sup[(df_long_tot.str_trans_UD_sup == inc1) | (df_long_tot.str_trans_UD_sup == inc2)]

            temp = repeating_stuff(vec0_vals, vec1_vals, vec2_vals, vec3_vals, vec4_vals, vec5_vals, incon)
            temp = temp.rename({0: 'str', 1: 'vals', 2: 'res_type'}, axis=1)
            
            #condensed_df_sup_2cat = pd.concat([condensed_df_sup_2cat, temp], axis=0)
            # OR
            condensed_df_sup_2cat = condensed_df_sup_2cat.append(temp)
            
            
            condensed_df_sup_2cat.reset_index(drop=True, inplace=True)
            condensed_df_sup_2cat = condensed_df_sup_2cat.rename({0: 'str', 1: 'vals', 2: 'res_type'}, axis=1)

In [ ]:
condensed_df_sup_2cat

### TR Plotting DataFrame

In [ ]:
# Plot of half the data
# incon_list = ['IC', 'EC', 'NC', 'NR']
incon_list = ['IC']


for ss in outer_name_list:
    if ss == 'sub':
        
        condensed_df_subTR = pd.DataFrame()

        for incon in incon_list:
            vec0_vals = df_long_totTR.vals_rot_RO_sub[(df_long_totTR.str_rot_RO_sub == incon)]
            vec1_vals = df_long_totTR.vals_rot_PI_sub[(df_long_totTR.str_rot_PI_sub == incon)]
            vec2_vals = df_long_totTR.vals_rot_YA_sub[(df_long_totTR.str_rot_YA_sub == incon)]
            vec3_vals = df_long_totTR.vals_trans_LR_sub[(df_long_totTR.str_trans_LR_sub == incon)]
            vec4_vals = df_long_totTR.vals_trans_FB_sub[(df_long_totTR.str_trans_FB_sub == incon)]
            vec5_vals = df_long_totTR.vals_trans_UD_sub[(df_long_totTR.str_trans_UD_sub == incon)]
            
            temp = repeating_stuff(vec0_vals, vec1_vals, vec2_vals, vec3_vals, vec4_vals, vec5_vals, incon)
            
            temp = temp.rename({0: 'str', 1: 'vals', 2: 'res_type'}, axis=1)
            
            #condensed_df_subTR = pd.concat([condensed_df_subTR, temp], axis=0)
            # OR
            condensed_df_subTR = condensed_df_subTR.append(temp)
            
            print('shape of condensed_df_subTR: ', condensed_df_subTR.shape)
            
            condensed_df_subTR.reset_index(drop=True, inplace=True)
            condensed_df_subTR = condensed_df_subTR.rename({0: 'str', 1: 'vals', 2: 'res_type'}, axis=1)
            
                
    elif ss == 'sup':
            
        condensed_df_supTR = pd.DataFrame()
            
        for incon in incon_list:
            vec0_vals = df_long_totTR.vals_rot_RO_sup[(df_long_totTR.str_rot_RO_sup == incon)]
            vec1_vals = df_long_totTR.vals_rot_PI_sup[(df_long_totTR.str_rot_PI_sup == incon)]
            vec2_vals = df_long_totTR.vals_rot_YA_sup[(df_long_totTR.str_rot_YA_sup == incon)]
            vec3_vals = df_long_totTR.vals_trans_LR_sup[(df_long_totTR.str_trans_LR_sup == incon)]
            vec4_vals = df_long_totTR.vals_trans_FB_sup[(df_long_totTR.str_trans_FB_sup == incon)]
            vec5_vals = df_long_totTR.vals_trans_UD_sup[(df_long_totTR.str_trans_UD_sup == incon)]

            temp = repeating_stuff(vec0_vals, vec1_vals, vec2_vals, vec3_vals, vec4_vals, vec5_vals, incon)
            temp = temp.rename({0: 'str', 1: 'vals', 2: 'res_type'}, axis=1)
            
            #condensed_df_supTR = pd.concat([condensed_df_supTR, temp], axis=0)
            # OR
            condensed_df_supTR = condensed_df_supTR.append(temp)
            
            
            condensed_df_supTR.reset_index(drop=True, inplace=True)
            condensed_df_supTR = condensed_df_supTR.rename({0: 'str', 1: 'vals', 2: 'res_type'}, axis=1)

In [ ]:
# Plot : For time response it is irrelavant to plot time for when they never got it correct : 'NC+NR'
incon_list = ['IC+EC']

for ss in outer_name_list:
    if ss == 'sub':
        
        condensed_df_subTR_2cat = pd.DataFrame()

        for incon in incon_list:
            if incon == 'IC+EC':
                inc1 = 'IC'
                inc2 = 'EC'
            elif incon == 'NC+NR':
                inc1 = 'NC'
                inc2 = 'NR'

            vec0_vals = df_long_totTR.vals_rot_RO_sub[(df_long_totTR.str_rot_RO_sub == inc1) | (df_long_totTR.str_rot_RO_sub == inc2)]
            vec1_vals = df_long_totTR.vals_rot_PI_sub[(df_long_totTR.str_rot_PI_sub == inc1) | (df_long_totTR.str_rot_PI_sub == inc2)]
            vec2_vals = df_long_totTR.vals_rot_YA_sub[(df_long_totTR.str_rot_YA_sub == inc1) | (df_long_totTR.str_rot_YA_sub == inc2)]
            vec3_vals = df_long_totTR.vals_trans_LR_sub[(df_long_totTR.str_trans_LR_sub == inc1) | (df_long_totTR.str_trans_LR_sub == inc2)]
            vec4_vals = df_long_totTR.vals_trans_FB_sub[(df_long_totTR.str_trans_FB_sub == inc1) | (df_long_totTR.str_trans_FB_sub == inc2)]
            vec5_vals = df_long_totTR.vals_trans_UD_sub[(df_long_totTR.str_trans_UD_sub == inc1) | (df_long_totTR.str_trans_UD_sub == inc2)]
            
            temp = repeating_stuff(vec0_vals, vec1_vals, vec2_vals, vec3_vals, vec4_vals, vec5_vals, incon)
            
            temp = temp.rename({0: 'str', 1: 'vals', 2: 'res_type'}, axis=1)
            
            #condensed_df_subTR_2cat = pd.concat([condensed_df_subTR_2cat, temp], axis=0)
            # OR
            condensed_df_subTR_2cat = condensed_df_subTR_2cat.append(temp)
            
            print('shape of condensed_df_subTR_2cat: ', condensed_df_subTR_2cat.shape)
            
            condensed_df_subTR_2cat.reset_index(drop=True, inplace=True)
            condensed_df_subTR_2cat = condensed_df_subTR_2cat.rename({0: 'str', 1: 'vals', 2: 'res_type'}, axis=1)
            
                
    elif ss == 'sup':
            
        condensed_df_supTR_2cat = pd.DataFrame()
            
        for incon in incon_list:
            if incon == 'IC+EC':
                inc1 = 'IC'
                inc2 = 'EC'
            elif incon == 'NC+NR':
                inc1 = 'NC'
                inc2 = 'NR'

            vec0_vals = df_long_totTR.vals_rot_RO_sup[(df_long_totTR.str_rot_RO_sup == inc1) | (df_long_totTR.str_rot_RO_sup == inc2)]
            vec1_vals = df_long_totTR.vals_rot_PI_sup[(df_long_totTR.str_rot_PI_sup == inc1) | (df_long_totTR.str_rot_PI_sup == inc2)]
            vec2_vals = df_long_totTR.vals_rot_YA_sup[(df_long_totTR.str_rot_YA_sup == inc1) | (df_long_totTR.str_rot_YA_sup == inc2)]
            vec3_vals = df_long_totTR.vals_trans_LR_sup[(df_long_totTR.str_trans_LR_sup == inc1) | (df_long_totTR.str_trans_LR_sup == inc2)]
            vec4_vals = df_long_totTR.vals_trans_FB_sup[(df_long_totTR.str_trans_FB_sup == inc1) | (df_long_totTR.str_trans_FB_sup == inc2)]
            vec5_vals = df_long_totTR.vals_trans_UD_sup[(df_long_totTR.str_trans_UD_sup == inc1) | (df_long_totTR.str_trans_UD_sup == inc2)]

            temp = repeating_stuff(vec0_vals, vec1_vals, vec2_vals, vec3_vals, vec4_vals, vec5_vals, incon)
            temp = temp.rename({0: 'str', 1: 'vals', 2: 'res_type'}, axis=1)
            
            #condensed_df_supTR_2cat = pd.concat([condensed_df_supTR_2cat, temp], axis=0)
            # OR
            condensed_df_supTR_2cat = condensed_df_supTR_2cat.append(temp)
            
            
            condensed_df_supTR_2cat.reset_index(drop=True, inplace=True)
            condensed_df_supTR_2cat = condensed_df_supTR_2cat.rename({0: 'str', 1: 'vals', 2: 'res_type'}, axis=1)

### Final plot

In [ ]:
import itertools
from textwrap import wrap

fig, ax = plt.subplots(3, 2, figsize=(20,10))  # the right=sub, left=sup

sns.set(font_scale = 2) # default is without style and palette
#sns.color_palette("light:#90a4ae", as_cmap=True)  # Greys_d, light:#5A9

front_edge_color = "#362d17"  # '#5c7d83' #'#669999'#
back_edge_color = "#362d17"  # black

alpha = 0.8

# ----------------------------
# Summed normalized count
# ----------------------------
po = 0
#sub: count
# IC+EC and NC+NR
oo = sns.barplot(x="str", y="vals", hue="res_type", data=condensed_df_sub_2cat, ax=ax[po,0], ci="sd", capsize=.1, estimator=np.sum, palette="light:#90a4ae", errcolor=back_edge_color, linewidth=3, edgecolor=back_edge_color, alpha=alpha, linestyle='-')

num_locations = len(condensed_df_sub_2cat)
hatches = itertools.cycle(['/', '//', '+', '-', 'x', '\\', '*', 'o', 'O', '.'])
for i, bar in enumerate(oo.patches):
    if i % num_locations == 0:
        hatch = next(hatches)
    bar.set_hatch(hatch)

# IC, NC
sns.barplot(x="str", y="vals", hue="res_type", data=condensed_df_sub, ax=ax[po,0], ci="sd", capsize=.1, estimator=np.sum, palette="light:#90a4ae", errcolor=front_edge_color, linewidth=3, edgecolor=front_edge_color, alpha=alpha, linestyle='-')

ax[po,0].set(ylim=(0, 150))
ax[po,0].set_ylabel("\n".join(wrap('Normalized count', 11)))  # left
ax[po,0].set_xlabel('', fontsize=22)

# ax[cou].set_yticks([])
# Remove legend
ax[po,0].legend([],[], frameon=False)

# Put the legend out of the figure
# ax[po,0].legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

# ----------------------------

#sup: count
# IC+EC and NC+NR
oo = sns.barplot(x="str", y="vals", hue="res_type", data=condensed_df_sup_2cat, ax=ax[po,1], ci="sd", capsize=.1, estimator=np.sum, palette="light:#90a4ae", errcolor=back_edge_color, linewidth=3, edgecolor=back_edge_color, alpha=alpha)

num_locations = len(condensed_df_sub_2cat)
hatches = itertools.cycle(['/', '//', '+', '-', 'x', '\\', '*', 'o', 'O', '.'])
for i, bar in enumerate(oo.patches):
    if i % num_locations == 0:
        hatch = next(hatches)
    bar.set_hatch(hatch)

# IC, NC
sns.barplot(x="str", y="vals", hue="res_type", data=condensed_df_sup, ax=ax[po,1], ci="sd", capsize=.1, estimator=np.sum, palette="light:#90a4ae", errcolor=front_edge_color, linewidth=3, edgecolor=front_edge_color, alpha=alpha)

ax[po,1].set(ylim=(0, 150))
ax[po,1].set_ylabel('', fontsize=22)  # right
ax[po,1].set_xlabel('', fontsize=22)

# Remove legend
ax[po,1].legend([],[], frameon=False)

# Put the legend out of the figure
# ax[po,1].legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
# ----------------------------

# Lower tail of 95 percent confidence interval per axis
ro_sub = condensed_df_sub_2cat.vals[(condensed_df_sub_2cat.str == 'RO')].to_numpy()
ro_sup = condensed_df_sup_2cat.vals[(condensed_df_sup_2cat.str == 'RO')].to_numpy()
pi_sub = condensed_df_sub_2cat.vals[(condensed_df_sub_2cat.str == 'PI')].to_numpy()
pi_sup = condensed_df_sup_2cat.vals[(condensed_df_sup_2cat.str == 'PI')].to_numpy()
ya_sub = condensed_df_sub_2cat.vals[(condensed_df_sub_2cat.str == 'YA')].to_numpy()
ya_sup = condensed_df_sup_2cat.vals[(condensed_df_sup_2cat.str == 'YA')].to_numpy()
lr_sub = condensed_df_sub_2cat.vals[(condensed_df_sub_2cat.str == 'LR')].to_numpy()
lr_sup = condensed_df_sup_2cat.vals[(condensed_df_sup_2cat.str == 'LR')].to_numpy()
fb_sub = condensed_df_sub_2cat.vals[(condensed_df_sub_2cat.str == 'FB')].to_numpy()
fb_sup = condensed_df_sup_2cat.vals[(condensed_df_sup_2cat.str == 'FB')].to_numpy()
ud_sub = condensed_df_sub_2cat.vals[(condensed_df_sub_2cat.str == 'UD')].to_numpy()
ud_sup = condensed_df_sup_2cat.vals[(condensed_df_sup_2cat.str == 'UD')].to_numpy()

# RO
linestyle = '-'
vec = make_a_properlist(np.concatenate((ro_sub, ro_sup), axis=0))
mean_dat, lower_tail, upper_tail = confidence_interval(vec, desired_CI=0.95)
ax[po,0].axhline(lower_tail, xmin=0, xmax=0.16, ls=linestyle, linewidth=3, color='r')
ax[po,1].axhline(lower_tail, xmin=0, xmax=0.16, ls=linestyle, linewidth=3, color='r')

# PI
vec = make_a_properlist(np.concatenate((pi_sub, pi_sup), axis=0))
mean_dat, lower_tail, upper_tail = confidence_interval(vec, desired_CI=0.95)
ax[po,0].axhline(lower_tail, xmin=0.18, xmax=0.33, ls=linestyle, linewidth=3, color='r')
ax[po,1].axhline(lower_tail, xmin=0.18, xmax=0.33, ls=linestyle, linewidth=3, color='r')

# YA
vec = make_a_properlist(np.concatenate((ya_sub, ya_sup), axis=0))
mean_dat, lower_tail, upper_tail = confidence_interval(vec, desired_CI=0.95)
ax[po,0].axhline(y=lower_tail, xmin=0.34, xmax=0.49, ls=linestyle, linewidth=3, color='r')
ax[po,1].axhline(y=lower_tail, xmin=0.34, xmax=0.49, ls=linestyle, linewidth=3, color='r')

# LR
vec = make_a_properlist(np.concatenate((lr_sub, lr_sup), axis=0))
mean_dat, lower_tail, upper_tail = confidence_interval(vec, desired_CI=0.95)
ax[po,0].axhline(y=lower_tail, xmin=0.51, xmax=0.66, ls=linestyle, linewidth=3, color='r')
ax[po,1].axhline(y=lower_tail, xmin=0.51, xmax=0.66, ls=linestyle, linewidth=3, color='r')

# FB
vec = make_a_properlist(np.concatenate((fb_sub, fb_sup), axis=0))
mean_dat, lower_tail, upper_tail = confidence_interval(vec, desired_CI=0.95)
ax[po,0].axhline(y=lower_tail, xmin=0.68, xmax=0.83, ls=linestyle, linewidth=3, color='r')
ax[po,1].axhline(y=lower_tail, xmin=0.68, xmax=0.83, ls=linestyle, linewidth=3, color='r')

# UD
vec = make_a_properlist(np.concatenate((ud_sub, ud_sup), axis=0))
mean_dat, lower_tail, upper_tail = confidence_interval(vec, desired_CI=0.95)
ax[po,0].axhline(y=lower_tail, xmin=0.84, xmax=1.0, ls=linestyle, linewidth=3, color='r')
ax[po,1].axhline(y=lower_tail, xmin=0.84, xmax=1.0, ls=linestyle, linewidth=3, color='r')

# ----------------------------
# Mean normalized count
# ----------------------------
po = 1
#sub: count
# IC+EC and NC+NR
oo = sns.barplot(x="str", y="vals", hue="res_type", data=condensed_df_sub_2cat, ax=ax[po,0], ci="sd", capsize=.1, estimator=np.mean, palette="light:#90a4ae", errcolor=back_edge_color, linewidth=3, edgecolor=back_edge_color, alpha=alpha)

num_locations = len(condensed_df_sub_2cat)
hatches = itertools.cycle(['/', '//', '+', '-', 'x', '\\', '*', 'o', 'O', '.'])
for i, bar in enumerate(oo.patches):
    if i % num_locations == 0:
        hatch = next(hatches)
    bar.set_hatch(hatch)

# IC, NC
sns.barplot(x="str", y="vals", hue="res_type", data=condensed_df_sub, ax=ax[po,0], ci="sd", capsize=.1, estimator=np.mean, palette="light:#90a4ae", errcolor=front_edge_color, linewidth=3, edgecolor=front_edge_color, alpha=alpha)

ax[po,0].set(ylim=(0, 12))
ax[po,0].set_ylabel("\n".join(wrap('Normalized Mean count', 11)))  # left
ax[po,0].set_xlabel('', fontsize=22)

# Remove legend
ax[po,0].legend([],[], frameon=False)

# Put the legend out of the figure
# ax[po,0].legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

# ----------------------------
#sup: count
# IC+EC and NC+NR
oo = sns.barplot(x="str", y="vals", hue="res_type", data=condensed_df_sup_2cat, ax=ax[po,1], ci="sd", capsize=.1, estimator=np.mean, palette="light:#90a4ae", errcolor=back_edge_color, linewidth=3, edgecolor=back_edge_color, alpha=alpha)

num_locations = len(condensed_df_sup_2cat)
hatches = itertools.cycle(['/', '//', '+', '-', 'x', '\\', '*', 'o', 'O', '.'])
for i, bar in enumerate(oo.patches):
    if i % num_locations == 0:
        hatch = next(hatches)
    bar.set_hatch(hatch)

# IC, NC
sns.barplot(x="str", y="vals", hue="res_type", data=condensed_df_sup, ax=ax[po,1], ci="sd", capsize=.1, estimator=np.mean, palette="light:#90a4ae", errcolor=front_edge_color, linewidth=3, edgecolor=front_edge_color, alpha=alpha)

ax[po,1].set(ylim=(0, 12))
ax[po,1].set_ylabel('', fontsize=22)  # right
ax[po,1].set_xlabel('', fontsize=22)

# Remove legend
# ax[po,1].legend([],[], frameon=False)

# Put the legend out of the figure
ax[po,1].legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., fontsize=17)

# ----------------------------



# ----------------------------
# Mean RT
# ----------------------------
po = 2
#sub: RT
# IC+EC
oo = sns.barplot(x="str", y="vals", hue="res_type", data=condensed_df_subTR_2cat, ax=ax[po,0], ci="sd", capsize=.1, estimator=np.mean, palette="light:#90a4ae", errcolor=back_edge_color, linewidth=3, edgecolor=back_edge_color, alpha=alpha)

num_locations = len(condensed_df_sub_2cat)
hatches = itertools.cycle(['/', '//', '+', '-', 'x', '\\', '*', 'o', 'O', '.'])
for i, bar in enumerate(oo.patches):
    if i % num_locations == 0:
        hatch = next(hatches)
    bar.set_hatch(hatch)

# IC
sns.barplot(x="str", y="vals", hue="res_type", data=condensed_df_subTR, ax=ax[po,0], ci="sd", capsize=.1, estimator=np.mean, palette="light:#90a4ae", errcolor=front_edge_color, linewidth=3, edgecolor=front_edge_color, alpha=alpha)

ax[po,0].set(ylim=(0, 13))
ax[po,0].set_ylabel("\n".join(wrap('Mean RT (s)', 11)))  # left
ax[po,0].set_xlabel('sub', fontsize=22)

# Remove legend
ax[po,0].legend([],[], frameon=False)

# Put the legend out of the figure
# ax[po,0].legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

# ----------------------------
#sup: RT
# IC+EC
oo = sns.barplot(x="str", y="vals", hue="res_type", data=condensed_df_supTR_2cat, ax=ax[po,1], ci="sd", capsize=.1, estimator=np.mean, palette="light:#90a4ae", errcolor=back_edge_color, linewidth=3, edgecolor=back_edge_color, alpha=alpha)

num_locations = len(condensed_df_sup_2cat)
hatches = itertools.cycle(['/', '//', '+', '-', 'x', '\\', '*', 'o', 'O', '.'])
for i, bar in enumerate(oo.patches):
    if i % num_locations == 0:
        hatch = next(hatches)
    bar.set_hatch(hatch)

# IC
sns.barplot(x="str", y="vals", hue="res_type", data=condensed_df_supTR, ax=ax[po,1], ci="sd", capsize=.1, estimator=np.mean, palette="light:#90a4ae", errcolor=front_edge_color, linewidth=3, edgecolor=front_edge_color, alpha=alpha)

ax[po,1].set(ylim=(0, 13))
ax[po,1].set_ylabel('', fontsize=22)  # right
ax[po,1].set_xlabel('sup', fontsize=22)

# Remove legend
ax[po,1].legend([],[], frameon=False)

# Put the legend out of the figure
# ax[po,1].legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
# ----------------------------  


plt.xticks(fontsize=22)

plt.savefig('Final_count_TR_rot_trans.png')